ps7.py, a skeleton of the solution

ps7_test.py, a test suite that will help you check your answers

triggers.txt, a sample trigger configuration file. You may modify this file to try other trigger configurations

feedparser.py, a module that will retrieve and parse feeds for you

project_util.py, a module that includes a function to convert simple HTML fragments to plain text

Conflict with Grader

To avoid getting an error from the grader about using the class variables title, subject, or summary, rename any variable you have used with these names to something else. Thank you!

PART 1: DATA STRUCTURE DESIGN  (5 points possible)
First, let's talk about one specific RSS feed: Google News. The URL for the Google News feed is:

http://news.google.com/?output=rss

If you try to load this URL in your browser, you'll probably see your browser's interpretation of the XML code generated by the feed. You can view the XML source with your browser's "View Page Source" function, though it probably will not make much sense to you. Abstractly, whenever you connect to the Google News RSS feed, you receive a list of items. Each entry in this list represents a single news item. In a Google News feed, every entry has the following fields:

guid : A globally unique identifier for this news story.

title : The news story's headline.

subject : A subject tag for this story (e.g. 'Top Stories', or 'Sports').

summary : A paragraph or so summarizing the news story.

link : A link to a web-site with the entire story.

Generalizing the Problem
This is a little trickier than we'd like it to be, because each of these RSS feeds is structured a little bit differently than the others. So, our goal in Part I is to come up with a unified, standard representation that we'll use to store a news story.

Why do we want this? When all is said and done, we want an application that aggregates several RSS feeds from various sources and can act on all of them in the exact same way: we should be able to read news stories from various RSS feeds all in one place. If you've ever used an RSS feed reader, be assured that it has had to solve the exact problem we're going to tackle in this pset!

PROBLEM 1

Parsing is the process of turning a data stream into a structured format that is more convenient to work with. We have provided you with code that will retrieve and parse the Google and Yahoo news feeds.

Parsing all of this information from the feeds that Google/Yahoo/the New York Times/etc. gives us is no small feat. So, let's tackle an easy part of the problem first: Pretend that someone has already done the specific parsing, and has left you with variables that contain the following information for a news story:

globally unique identifier (GUID) - a string that serves as a unique name for this entry

title - a string

subject - a string

summary - a string

link to more content - a string

We want to store this information in an object that we can then pass around in the rest of our program. Your task, in this problem, is to write a class, NewsStory, starting with a constructor that takes (guid, title, subject, summary, link) as arguments and stores them appropriately. NewsStory also needs to contain the following methods:

getGuid(self)
getTitle(self)
getSubject(self)
getSummary(self)
getLink(self)
Each method should return the appropriate element of an instance. For example, if we have implemented the class and call

test = NewsStory('foo', 'myTitle', 'mySubject', 'some long summary', 'www.example.com')
then test.getGuid() will return foo.
The solution to this problem should be relatively short and very straightforward (please review what get methods should do if you find yourself writing multiple lines of code for each). Once you have implemented NewsStory all the NewsStory test cases should work.

To test your class definition, we have provided a test suite in ps7_test.py. You can test your code by loading and running this file. You should see an "OK" for the NewsStory tests if your code is correct. Because ps7.py contains code to run the full RSS scraping system, we suggest you do not try to run ps7.py directly to test your implementation. Instead, in IDLE, you can do the following:

```
>>> from ps7 import *
>>> test = ps7.NewsStory('foo', 'myTitle', 'mySubject', 'some long summary', 'www.example.com')
to load in then run your own tests on your class definitions.
Canopy Specific Instructions: If you are getting an error, type the following instead:
>>> cd [insert the full path of the directory where your code resides]
>>> from ps7 import *
>>> test = NewsStory('foo', 'myTitle', 'mySubject', 'some long summary', 'www.example.com')
```

In [17]:
# Enter your code for NewsStory in this box
class NewsStory(object):
    def __init__(self, guid, title, subject, summary, link):
        self.guid = guid
        self.title = title
        self.subject = subject
        self.summary = summary
        self.link = link
    def getGuid(self):
        return self.guid
    def getTitle(self):
        return self.title
    def getSubject(self):
        return self.subject
    def getSummary(self):
        return self.summary
    def getLink(self):
        return self.link
test = NewsStory('foo', 'myTitle', 'mySubject', 'some long summary', 'www.example.com')
test.getTitle()

'myTitle'

PART 2: TRIGGERS (WORDTRIGGERS)  (20 points possible)
Given a set of news stories, your program will generate alerts for a subset of those stories. Stories with alerts will be displayed to the user, and the other stories will be silently discarded. We will represent alerting rules as triggers. A trigger is a rule that is evaluated over a single news story and may fire to generate an alert. For example, a simple trigger could fire for every news story whose title contained the word "Microsoft". Another trigger may be set up to fire for all news stories where the summary contained the word "Boston". Finally, a more specific trigger could be set up to fire only when a news story contained both the words "Microsoft" and "Boston" in the summary.

In order to simplify our code, we will use object polymorphism. We will define a trigger interface and then implement a number of different classes that implement that trigger interface in different ways.

TRIGGER INTERFACE

Each trigger class you define should implement the following interface, either directly or transitively. It must implement the evaluate method that takes a news item (NewsStory object) as an input and returns True if an alert should be generated for that item. We will not directly use the implementation of the Trigger class, which is why it throws an exception should anyone attempt to use it

The class below implements the Trigger interface (you will not modify this). Any subclass that inherits from it will have an evaluate method. By default, they will use the evaluate method in Trigger, the superclass, unless they define their own evaluate function, which would then be used instead. If some subclass neglects to define its own evaluate() method, calls to it will go to Trigger.evaluate(), which fails (albeit cleanly) with the NotImplementedError exception:

class Trigger(object):
    def evaluate(self, story):
        """
        Returns True if an alert should be generated
        for the given news item, or False otherwise.
        """
        raise NotImplementedError
We will define a number of classes that inherit from Trigger. In the figure below, Trigger is a superclass, which all other classes inherit from. The arrow from WordTrigger to Trigger means that WordTrigger inherits from Trigger - a WordTrigger is a Trigger . Note that other classes inherit from WordTrigger.

 

[Click on the above image for a full-size view]

WHOLE WORD TRIGGERS

Having a trigger that always fires isn't interesting; let's write some that are. A user may want to be alerted about news items that contain specific words. For instance, a simple trigger could fire for every news item whose title contains the word "Microsoft". In the following problems, you will create a WordTrigger abstract class and implement three classes that inherit from this class.

The trigger should fire when the whole word is present. For example, a trigger for "soft" should fire on:

Koala bears are soft and cuddly.

I prefer pillows that are soft.

Soft drinks are great.

Soft's the new pink!

"Soft!" he exclaimed as he threw the football.

But should not fire on:

Microsoft recently released the Windows 8 Consumer Preview.

Downey makes my clothes the softest they can be!

This is a little tricky, especially the case with the apostrophe. For the purpose of your parsing, pretend that a space or any character in string.punctuation is a word separator. If you've never seen string.punctuation before, go to your interpreter and type:
>>> import string
>>> print string.punctuation
Play around with this a bit to get comfortable with what it is. The split and replace methods of strings will almost certainly be helpful as you tackle this part.
You may also find the string methods lower and/or upper useful for this problem.

PROBLEM 2

Implement a word trigger abstract class, WordTrigger. It should take in a string word as an argument to the class's constructor.

WordTrigger should be a subclass of Trigger. It has one new method, isWordIn, which takes in one string argument text. It returns True if the whole word word is present in text, False otherwise, as described in the above examples. This method should not be case-sensitive. Implement this method.

Hint

Because this is an abstract class, we will not be directly instantiating any WordTriggers. WordTrigger should inherit its evaluate method from Trigger. We do this because now we can create subclasses of WordTrigger that use its isWordIn method. In this way, it is much like the Trigger interface, except now actual code from this WordTrigger class is used in its subclasses.

PROBLEM 3

You are now ready to implement WordTrigger's three subclasses: TitleTrigger, SubjectTrigger, and SummaryTrigger.

Implement a word trigger class, TitleTrigger, that fires when a news item's title contains a given word. The word should be an argument to the class's constructor. This trigger should not be case-sensitive (it should treat "Intel" and "intel" as being equal).

For example, an instance of this type of trigger could be used to generate an alert whenever the word "Intel" occurred in the title of a news item. Another instance could generate an alert whenever the word "Microsoft" occurred in the title of an item.

Think carefully about what methods should be defined in TitleTrigger and what methods should be inherited from the superclass. This class can be implemented in as few as 3 lines code!

Hint

Once you've implemented TitleTrigger, the TitleTrigger unit tests in our test suite should pass. Run ps7_test.py to check.

Canopy specific instructions:Every time you modify code inps7.pygo to
Run -> Restart Kernel (or hit the CTRL with the dot on your keyboard)
before running ps7_test.py. You have to do this every time you modify the fileps7.pyand want to run the fileps7_test.py, otherwise changes to the former will not be incorporated in the latter.

PROBLEM 4

Implement a word trigger class, SubjectTrigger, that fires when a news item's subject contains a given word. The word should be an argument to the class's constructor. This trigger should not be case-sensitive.

Once you've implemented SubjectTrigger, the SubjectTrigger unit tests in our test suite should pass.

PROBLEM 5

Implement a word trigger class, SummaryTrigger, that fires when a news item's summary contains a given word. The word should be an argument to the class's constructor. This trigger should not be case-sensitive.

Once you've implemented SummaryTrigger, the SummaryTrigger unit tests in our test suite should pass.

In [4]:
import string
print string.punctuation

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [5]:
"safass*ss_".split(string.punctuation)

['safass*ss_']

In [16]:
a = "safass*ss_"
sep = string.punctuation
for char in sep:
    #print char
    a = a.replace( str(char), ' ')
print a  
b=a.split(' ')
print b

safass ss 
['safass', 'ss', '']


In [ ]:
class WordTrigger(Trigger):
    def __init__(self, word):
        self.word = word
    
    def isWordIn(self, text):
        sep = string.punctuation
        for char in sep:
            #print char
            text = text.replace(char, ' ')
        wordsList = text.split(' ')
        lowerWordsList = []        
        for word in wordsList:
            lowerWordsList.append(word.lower())
        word = self.word
        word = word.lower()
        if word in wordsList:
            return True
        else:
            return False

# TODO: TitleTrigger
def TitleTrigger(WordTrigger):
    def evaluate(self, story):
        return self.isWordIn(self.getTitle(), story)    
        
# TODO: SubjectTrigger
def SubjectTrigger(WordTrigger):
    def evaluate(self, story):
        return self.isWordIn(self.getSubject(), story)
        
# TODO: SummaryTrigger
def SummaryTrigger(WordTrigger):
    def evaluate(self, story):
        return self.isWordIn(self.getSummary(), story)
